<a href="https://colab.research.google.com/github/MillyLiu87/Info/blob/main/Milvus_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [137]:
!pip install openai requests tqdm


In [138]:
pip install grpcio==1.63.0

# We will use OpenAI as the LLM in this example. You should prepare the api key OPENAI_API_KEY as an environment variable.

In [139]:
import os

os.environ["OPENAI_API_KEY"] = "sk"


# 明白老师的38节写作基础课，文字稿

In [140]:
import json
import urllib.request

file_path = "Mingbai_writing.txt"

if not os.path.exists(file_path):
    urllib.request.urlretrieve(url, file_path)


# 按照章节进行拆分，也就是Markdown的"# "符号

In [141]:
with open(file_path, "r") as file:
    file_text = file.read()

text_lines = file_text.split("# ")



In [144]:
from openai import OpenAI

openai_client = OpenAI()

def emb_text(text):
    return (
        openai_client.embeddings.create(input=text, model="text-embedding-3-small")
        .data[0]
        .embedding
    )


In [145]:
test_embedding = emb_text("This is a test")
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])

1536
[0.00988506618887186, -0.005540902726352215, 0.0068014683201909065, -0.03810417652130127, -0.018254263326525688, -0.041231658309698105, -0.007651153020560741, 0.03220026567578316, 0.01892443746328354, 0.00010708322952268645]


# 创建数据库Collection

In [146]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./milvus_demo.db")

collection_name = "my_rag_collection"



DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: cf7bee65e7a346afaf004db8b9a2caed


# 如果Collection已经存在，则丢弃





In [147]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)


如果我们不指定任何字段信息,Milvus会自动创建以下内容:
一个默认的id字段作为主键
一个向量字段用于存储向量数据
一个保留的JSON字段,用于存储未在schema中定义的字段及其值
这种自动创建的默认结构为用户提供了便利,无需手动定义每个字段就可以快速开始使用Milvus。同时,保留的JSON字段也提供了灵活性,允许存储一些额外的、未在schema中预先定义的数据。这对于处理动态或不确定的数据结构非常有用。

In [148]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # Inner product distance
    consistency_level="Strong",  # Strong consistency level
)


DEBUG:pymilvus.milvus_client.milvus_client:Successfully created collection: my_rag_collection
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created an index on collection: my_rag_collection


遍历文本行,创建嵌入向量,然后将数据插入到Milvus中。
这里有一个新字段text,它在集合模式中是未定义的字段。它将被自动添加到保留的JSON动态字段中,在高层次上可以被视为一个普通字段。
这段话解释了以下几点:
处理数据的过程:遍历文本,为每行创建嵌入向量,然后将数据插入Milvus。
引入了一个新字段"text",这个字段在原始的集合模式(schema)中并未定义。
尽管"text"字段未在模式中定义,但由于启用了动态字段功能,它会被自动添加到一个保留的JSON动态字段中。
虽然"text"是动态添加的字段,但在使用时可以像普通预定义字段一样对待它。
这体现了Milvus的动态模式特性,允许在不修改原始模式的情况下灵活地添加新字段,提高了数据插入和管理的灵活性。

In [149]:
from tqdm import tqdm

data = []

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": emb_text(line), "text": line})

milvus_client.insert(collection_name=collection_name, data=data)


Creating embeddings: 100%|██████████| 39/39 [00:16<00:00,  2.43it/s]


{'insert_count': 39,
 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38],
 'cost': 0}

In [150]:
question = "为什么要写作"


在集合中搜索问题并检索语义上最相似的前3个匹配结果。
这段代码的具体含义是：
使用milvus_client.search方法在指定的集合中进行搜索。
搜索的数据是将输入的问题(question)通过emb_text函数转换成的嵌入向量。
limit=3表示返回相似度最高的前3个结果。
搜索参数使用内积（Inner Product, IP）作为相似度度量方法。
output_fields=["text"]指定返回结果中包含"text"字段。
这种搜索方法利用了向量相似度来找到语义上最接近的匹配，而不是简单的关键词匹配，因此能够检索到语义上相关的结果。

In [151]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=[
        emb_text(question)
    ],  # Use the `emb_text` function to convert the question to an embedding vector
    limit=3,  # Return top 3 results
    search_params={"metric_type": "IP", "params": {}},  # Inner product distance
    output_fields=["text"],  # Return the text field
)


In [152]:
retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))


[
    [
        "\u5199\u4f5c\u57fa\u7840\u7b2c07\u8bfe\uff1a \u666e\u901a\u4eba\u8ba4\u8bc6\u5199\u4f5c\u7684 4 \u4e2a\u9636\u6bb5\n\u6211\u575a\u6301\u8ba4\u4e3a\uff0c\u5bf9\u666e\u901a\u4eba\u6765\u8bf4\uff0c\u5199\u4f5c\u662f\u666e\u901a\u4eba\u8d5a\u94b1\u8def\u4e0a\u4e0d\u53ef\u6216\u7f3a\u7684\u4e00\u4e2a\u57fa\u7840\u6280\u80fd\u3002\n\u8fd9\u91cc\u4e0d\u662f\u8bf4\uff0c\u4f60\u8981\u8d5a\u94b1\uff0c\u5c31\u4e00\u5b9a\u8981\u5b66\u5199\u4f5c\uff0c\u4e0d\u5b66\u5199\u4f5c\u5c31\u4e00\u5b9a\u8d5a\u4e0d\u5230\u94b1\u3002\n\u800c\u662f\uff0c\u5199\u4f5c\u53ef\u4ee5\u8ba9\u6211\u4eec\u7684\u8d5a\u94b1\u6548\u7387\u3001\u6982\u7387\u5927\u5927\u63d0\u9ad8\u3002\n\u539f\u56e0\u5728\u524d\u9762 10-17 \u8282\u8bfe\u91cc\u8bb2\u8fc7\u3002\n\u4e8b\u5b9e\u4e0a\uff0c\u6211\u4eec\u4e5f\u80fd\u770b\u5230\uff0c\u8eab\u8fb9\u8d8a\u6765\u8d8a\u591a\u4eba\u5f00\u59cb\u5b66\u4e60\u5199\u4f5c\u4e86\u3002\n\u4e0d\u8fc7\uff0c\u6709\u4e9b\u4eba\u5b66\u4e60\u5199\u4f5c\uff0c\u4e0d\u662f\u771f\u7684\u74

In [153]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)


In [154]:
print(context)

写作基础第07课： 普通人认识写作的 4 个阶段
我坚持认为，对普通人来说，写作是普通人赚钱路上不可或缺的一个基础技能。
这里不是说，你要赚钱，就一定要学写作，不学写作就一定赚不到钱。
而是，写作可以让我们的赚钱效率、概率大大提高。
原因在前面 10-17 节课里讲过。
事实上，我们也能看到，身边越来越多人开始学习写作了。
不过，有些人学习写作，不是真的理解写作是什么，而是看到别人通过写作赚钱了，所以也去学习写作。
这种情况，大概率写不好。
今天就来分享一下，人们对写作的认识，分为 4 个阶段。
只有达到第 4 阶段，才算真的理解写作。
相反，如果不理解这点，就容易变成无效勤奋。
1 写作 = 输出
第 1 阶段的人，看到别人通过写作赚钱，于是也想学习写作。
他们觉得，写作主要做的事情，就是写。
只要自己不断写，就能提高写作能力，写出好文章，获得转发，然后涨粉，接广告、变现。
但结果是，写了 2 篇，发现没人看，没人关注，没人转发，赚不到钱，于是放弃。
更严重的是，有的人会因此觉得：「我不适合写作」，甚至有人会觉得：我好差，为什么别人「轻轻松松」就能写出 10w+，我却做不好。他们会自我批判，产生挫败情绪，觉得自己学什么都学不好，陷入习得性无助。
这是普通人对写作认识的第 1 阶段。
2 写作 = 输入 + 输出
第 2 阶段的人，比第 1 阶段的人多一些思考。
他们会想，既然要写，那写什么呢？我没啥东西可写啊！对了，我读一些书，然后写出来就好了。于是，他们会去读一些书，然后写读书笔记、心得等。
他们觉得只要自己多读书，然后把书里的内容写出来，就会有价值，写出好文章，获得转发，然后涨粉，接广告、变现。
但结果是，写了 5 篇，发现还是没人看，赚不到钱，于是放弃。
他们觉得，我都读书了，写出来的文章还没人看，说明我不适合写作。那些能写出好文章的人，应该是凭天赋吧。我没有天赋，嗯，算了。
这是普通人对写作认识的第 2 阶段。
3 写作 = 输入 + 思考 + 输出
第 3 阶段的人，比第 2 阶段多一些思考。
他们会去学习一些产品、定位、营销的知识，知道要分析目标用户，要选择写作话题等。
他们也会听写作课、读写作书籍、学习写作技巧，吸取高手的经验来帮助自己提高写作水平。
但是，他们的进步比较慢。
因为，虽然他们听课、读书，学习各种写作方法。但他们会忽略了一个因素：积累。

这个用户提示为语言模型提供了具体的指令:
它指示模型使用<context>标签中的信息来回答问题。
{context}是一个占位符,将被从Milvus检索到的相关文档填充。
{question}是一个占位符,将被用户的具体问题替换。
这种提示结构的设计目的是:
为语言模型定义系统级行为(作为能回答问题的AI助手)。
提供一个结构化的格式,将检索到的上下文信息和用户问题传递给模型。
指导模型使用提供的上下文来回答特定问题,而不是依赖其预训练知识。
这种方法结合了检索增强生成(RAG)技术,通过从Milvus检索相关文档来增强语言模型的回答能力。

In [155]:
SYSTEM_PROMPT = """
Human: You are an AI assistant. You are able to find answers to the questions from the contextual passage snippets provided.
"""
USER_PROMPT = f"""
Use the following pieces of information enclosed in <context> tags to provide an answer to the question enclosed in <question> tags. Reply in Chinese
<context>
{context}
</context>
<question>
{question}
</question>
"""


In [156]:
response = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)


写作对于普通人来说，是一个重要的基础技能。通过写作，可以提高赚钱效率和概率，促进个人成长和提升。通过深入思考写作的意义和价值，可以帮助人们更有方向、更有效率地写作，并从中获得更大的收益。写作不仅仅是一种技能，更是一个系统，需要输入、思考、输出等多个因素相结合。通过持续写作，一个人的能力和价值也会不断提高，最终实现个人成长和收益的目标。
